In [1]:
import torch
from torchsummary import summary
from model import DNNmodel
from torch.nn.utils.rnn import pad_packed_sequence,pack_sequence,pad_sequence

In [5]:
device = torch.device('mps')

In [2]:
mymodel = DNNmodel.DNNmodel()
summary(mymodel)

Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              16
├─Linear: 1-2                            2
├─Dropout: 1-3                           --
Total params: 18
Trainable params: 18
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              16
├─Linear: 1-2                            2
├─Dropout: 1-3                           --
Total params: 18
Trainable params: 18
Non-trainable params: 0

In [3]:
a = torch.randn((21000, 1, 1))
x = mymodel(a)

## Test if a single epoch works

In [4]:
from dataset import dataloader
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [5]:
"""
test case: 
singer id: ADIZ
song id: 01
"""

testclass = dataloader.ParallelData()
testdatainfo, testsingersplit = testclass.__getitem__("ADIZ")
testdatasplit = testsingersplit["01"] # 338, 21000
train_loader = DataLoader(testdatasplit, batch_size=32, shuffle=True, drop_last=True)

In [6]:
a = torch.Tensor(testdatasplit)
b = torch.reshape(a, (338, 21000, 1))
train_loader = DataLoader(b, batch_size=32, shuffle=True, drop_last=True)
for i, data in enumerate(train_loader):
    print(i, data.shape)

0 torch.Size([32, 21000, 1])
1 torch.Size([32, 21000, 1])
2 torch.Size([32, 21000, 1])
3 torch.Size([32, 21000, 1])
4 torch.Size([32, 21000, 1])
5 torch.Size([32, 21000, 1])
6 torch.Size([32, 21000, 1])
7 torch.Size([32, 21000, 1])
8 torch.Size([32, 21000, 1])
9 torch.Size([32, 21000, 1])


/var/folders/jr/tf9k36ws1956sfj0w8ybnj2m0000gn/T/ipykernel_73716/1752668894.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1675670936158/work/torch/csrc/utils/tensor_new.cpp:248.)
  a = torch.Tensor(testdatasplit)


In [7]:
traindata = data.transpose(0,1)
traindata.shape

torch.Size([21000, 32, 1])

In [8]:
testmodel = DNNmodel.DNNmodel(1, 16, 1, 1, 0.1)
x = mymodel(traindata)

## Test with epochs

In [11]:
import torch.nn as nn
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=1e-2)

In [13]:
testdatainfo2, testsingersplit2 = testclass.__getitem__("SAMF")
testdatasplit2 = testsingersplit2["01"] # 346, 21000
len(testdatasplit2) 

346

In [14]:
a2 = torch.Tensor(testdatasplit2)
b2 = torch.reshape(a2, (346, 21000, 1))
train_loader2 = DataLoader(b2, batch_size=32, shuffle=True, drop_last=True)
for i, data2 in enumerate(train_loader2):
    print(i, data2.shape)

0 torch.Size([32, 21000, 1])
1 torch.Size([32, 21000, 1])
2 torch.Size([32, 21000, 1])
3 torch.Size([32, 21000, 1])
4 torch.Size([32, 21000, 1])
5 torch.Size([32, 21000, 1])
6 torch.Size([32, 21000, 1])
7 torch.Size([32, 21000, 1])
8 torch.Size([32, 21000, 1])
9 torch.Size([32, 21000, 1])


In [15]:
max_epochs = 100
for epoch in range(max_epochs):
    output = mymodel(data)
    loss = loss_function(output, data2)
 
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
 
    if loss.item() < 1e-4:
        print('Epoch [{}/{}], Loss: {:.5f}'.format(epoch+1, max_epochs, loss.item()))
        print("The loss value is reached")
        break
    elif (epoch+1) % 10 == 0:
        print('Epoch: [{}/{}], Loss:{:.5f}'.format(epoch+1, max_epochs, loss.item()))
    

Epoch: [10/100], Loss:0.23357
Epoch: [20/100], Loss:0.13640
Epoch: [30/100], Loss:0.07718
Epoch: [40/100], Loss:0.04115
Epoch: [50/100], Loss:0.02099
Epoch: [60/100], Loss:0.01167
Epoch: [70/100], Loss:0.00861
Epoch: [80/100], Loss:0.00814
Epoch: [90/100], Loss:0.00820
Epoch: [100/100], Loss:0.00820


### predict

In [35]:
predictdatainfo, predictsingersplit = testclass.__getitem__("ADIZ")
predictdatasplit = predictsingersplit["09"] # 346, 21000
len(testdatasplit2) 

346

In [36]:
a3 = torch.Tensor(testdatasplit2)
b3 = torch.reshape(a3, (346, 21000, 1))
train_loader3 = DataLoader(b3, batch_size=32, shuffle=False, drop_last=True)
for i, data3 in enumerate(train_loader2):
    print(i, data3.shape)

0 torch.Size([32, 21000, 1])
1 torch.Size([32, 21000, 1])
2 torch.Size([32, 21000, 1])
3 torch.Size([32, 21000, 1])
4 torch.Size([32, 21000, 1])
5 torch.Size([32, 21000, 1])
6 torch.Size([32, 21000, 1])
7 torch.Size([32, 21000, 1])
8 torch.Size([32, 21000, 1])
9 torch.Size([32, 21000, 1])


In [37]:
predictive_result = mymodel(data3)
predictive_result.shape

torch.Size([32, 21000, 1])

In [38]:
c = predictive_result.view(-1, 1).data.numpy()
c.shape

(672000, 1)

### Audio

In [25]:
from IPython.display import Audio

In [39]:
Audio(data=c[:,0], rate=22050)